In [880]:
import pandas as pd
import math
import io
from datetime import datetime
from alive_progress import alive_bar


### Read Data

In [881]:
### Note: if using Google Colab, this must be set to the filename of the 
# file you will load. This is a quirk of Colab. 
filename = "2020-2022 Training Records.xlsx"

try:
  import google.colab
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  source = pd.read_excel(io.BytesIO(uploaded.get(filename)))
else:
  print("Not Colab")
  source = pd.read_excel(filename)

Not Colab


In [882]:
FILTER = "RANDOM"
SIZE = 1000

if FILTER == "RANDOM":
    source = source.sample(SIZE)
elif FILTER == "HEAD":
    source = source.head(SIZE)
elif FILTER == "TAIL":
    source = source.tail(SIZE)


In [883]:
source.head(15)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
148485,90525,"OFORI-OPOKU, Nana",NaN,nana.ofori-opoku@cnl.ca,Computational Materials Scient,2701.0,COMPUTATIONAL TECHNIQUES,NaN,Course,OSH-9045-Online (Rev 1),Cost Course - Hand Safety (1.0),1.0,100.0,Attended,Pass,Completed,2021-07-13,NaT,NaN
115515,CONT-2835,"MONTOYA, RAFAEL",ATOM #54424607,NaN,NaN,NaN,NaN,"BUDARICK, Jennifer",ILT Class,RP-G3,Group 3 - Radiation Protection Training,NaN,86.0,Attended,Pass,Completed,2021-05-03,NaT,NaN
122278,92648,"NIEMAN, Dominic",NaN,dominic.nieman@cnl.ca,"Supervisor, Aggregate/Spoils",8599.0,WASTE SUPPORT SERVICES,"DELOUGHERY, Patrick Mark",Versional,OPEX-1005-Online,Introducing the ImpAct Process at CNL,NaN,NaN,Attended,NaN,Completed,2022-02-08,NaT,NaN
71926,90595,"KILLEEN, Dwane",NaN,dwane.killeen@cnl.ca,Planner,8270.0,CRL DECOMM PROGRAM EXECUTION,"DEGAGNE, Pat",Versional,ENG-1069-Online,Fume Hood Safety,NaN,NaN,Attended,NaN,Completed,2021-10-12,2024-10-12,NaN
133345,56660,"HILL, Daniel",NaN,Dan.Hill@cnl.ca,Asset Manager,8400.0,LOGISTICS & ASSET MGMT,"CHARBONNEAU, Chad",Course,LGL-1003-Online (Rev 2.5),Values and Ethics at CNL (2.5),2.5,100.0,Attended,Pass,Completed,2021-04-13,2022-03-02,NaN
154157,91821,"BAGHERIPOOR, Mahdi",NaN,mahdi.bagheripoor@cnl.ca,New Grad - Material Scientist,1101.0,MATERIALS SCIENCE,"BASCHUK, Jeffrey",ILT Class,PROC-1001,Basic Oracle iProcurement Requisition Awareness,NaN,NaN,Attended,Pass,Completed,2021-09-22,NaT,NaN
1175,10845,"KITSON, Charles",NaN,Charles.Kitson@cnl.ca,Environmental Analyst,8588.0,WL ENVIRONMENTAL MANAGEMENT,"BETTERIDGE, James",Document,900-510000-MCP-015 (Rev 1),Harassment and Violance Free Workplace (1.0),1.0,NaN,Attended,NaN,Completed,2021-03-24,NaT,NaN
86910,91381,"HAYCOCK, Meghan",NaN,meghan.haycock@cnl.ca,Waste Characterization Special,8686.0,WASTE CHARACTERIZATION SERVICE,"CHARTRAND, Julie",Versional,OSH-1004-Online,Lock Out Tag Out Exam,NaN,NaN,Attended,Pass,Completed,2022-01-26,2025-01-26,NaN
128999,53894,"LOTT, Michael",NaN,Michael.Lott@cnl.ca,Firefighter,4173.0,CRL FIRE OPERATIONS,"BOXALL, Justin",ILT Course,FIRE-3058-CRL,Fire Stream: Practical: Operating and Maintain...,NaN,NaN,Attended,NaN,Completed,2021-06-10,2022-06-11,NaN
56731,60693,"LI, Kevin",NaN,kevin.li@cnl.ca,R&D Scientist,8171.0,FLUIDS ENGINEERING,"EYVINDSON, Ailsa",Course,FIRE-1001-Online (Rev 4.4),Fire Prevention (4.4),4.4,NaN,Cancelled,NaN,NaN,2021-03-25,NaT,NaN


In [884]:
print(source.shape[1])
source[["Completion Status", "Attendance Status", "Pass/Fail"]].groupby(["Pass/Fail","Completion Status", "Attendance Status"], dropna=False, as_index=False).size()

19


,Pass/Fail,Completion Status,Attendance Status,size
0,Fail,Completed,Attended,36
1,Fail,NaN,Attended,1
2,Pass,Completed,Attended,677
3,Pass,Completed,Waived,10
4,NaN,Completed,Attended,198
5,NaN,NaN,Attended,3
6,NaN,NaN,Cancelled,70
7,NaN,NaN,No-show,4
8,NaN,NaN,Replaced,1


### Read SABA Data

#### Course Data

In [885]:
courses = pd.read_csv("saba_courses.csv")

#subset = courses[["Course ID", "Version"]]
#tuples = [tuple(x) for x in subset.to_numpy()]

#course_id = set(tuples)
course_id = courses[["Course ID", "Version"]]
ids_only = pd.Series(courses["Course ID"].unique())
#course_id = set((courses["Course ID"], courses["Version"]))
#courses.head()
len(course_id)
#course_id

1457

In [886]:
("IT-9050", 1.0) in course_id

False

#### Employee Data

In [887]:
people = pd.read_csv("saba_people.csv")
people_un = set(people["Username"].astype("int64", errors="ignore").unique())
#people.head()
len(people_un)

3869

In [888]:
"91003" in people_un

True

### Process Data

In [889]:
def procStatus(pass_fail, comp_status, att_status):
    status = -1
    ## Fail
    if pass_fail == "Fail":
        status = 400
    elif (pass_fail == "Pass") & (comp_status == "Not Completed"):
        status = 400
    #elif (math.isnan(pass_fail)) & (math.isnan(comp_status)) & ((att_status == "No-show") | (att_status == "Replaced")):
    #    status = 400
    ## Pass
    elif (pass_fail == "Pass") & (comp_status != "Not Completed"):
        status = 200
    elif (pd.isnull(pass_fail)) & (comp_status == "Completed"):
        status = 200
        #print("no p/f, status is complete")
    elif ( (pd.isnull(pass_fail)) & (pd.isnull(comp_status)) & (att_status == "Attended") ):
        status = 200
        #print("no p/f, no status, attstatus is attended")
    else:
        status = 400
    return status

def checkPerson(unList, person):
    is_in = False
    if str(person) in unList:
        is_in = True
    return is_in

def checkCourse(courseList, courseID, version):
    is_in = False
    replace_version = version
    for _, row in courseList.iterrows():
        #print(courseID, version, row["Course ID"], row["Version"])
        if (courseID == row["Course ID"]):
            #print("Course Match", courseID, version, row["Course ID"], row["Version"])
            replace_version = row["Version"]
            is_in = True

    return is_in, replace_version

In [890]:
# Helpers

class TranscriptRow:
    def __init__(self, username, courseID, courseVersion, compDate, status, courseTitle, inCourse, inPerson):
        self.username = username
        self.courseID = courseID
        self.courseVersion = courseVersion
        self.compDate = compDate
        #self.status = self.makeStatus(status)
        self.status = status
        self.courseTitle = courseTitle
        self.inCourse = inCourse
        self.inPerson = inPerson
    
    def makeStatus(self, status):
        newStat = -1
        if status == "Completed":
            newStat = 200
        return newStat

In [891]:
USERNAME = "Employee Number"
COURSEID = "OFFERING"
COURSEVERSION = "COURSE_VERSION"
COMPDATE = "Completion Date"
STATUS = "Completion Status"
COURSETITLE = "Activity Name"
PASSFAIL = "Pass/Fail"
ATTSTATUS = "Attendance Status"

source_used = source[[USERNAME, COURSEID, COURSEVERSION, COMPDATE, PASSFAIL, STATUS, ATTSTATUS, COURSETITLE]]

In [892]:
transcripts = []

with alive_bar(source_used.shape[0], force_tty=True) as bar_read:
    for index, trans in source_used.iterrows():
        username = trans[USERNAME]
        courseID = trans[COURSEID]
        course_ver = trans[COURSEVERSION]
        comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
        status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
        course_title = trans[COURSETITLE]
        in_course, rep_ver = checkCourse(course_id, trans[COURSEID], trans[COURSEVERSION])
        in_pers = checkPerson(people_un, trans[USERNAME])

        #Update Version
        course_ver = rep_ver
        
        trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
        transcripts.append(trans_entry)
        #print(courseID, type(courseID), courseID in ids_only.values)
        bar_read()
print(len(transcripts))

|████████████████████████████████████████| 1000/1000 [100%] in 1:05.3 (15.32/s)                                         a: 1:12)  ▄▂▂ 44/1000 [4%] in 3s (14.3/s, eta: 1:07)  ▃▅▇ 82/1000 [8%] in 5s (15.1/s, eta: 1:01) ▆▄▂ 134/1000 [13%] in 10s (13.9/s, eta: 1:02)                         ▆█▆ 237/1000 [24%] in 16s (14.4/s, eta: 53s)  ▅▃▁ 246/1000 [25%] in 17s (14.4/s, eta: 52s) ▃▁▃ 252/1000 [25%] in 17s (14.5/s, eta: 52s)  ▃▅▇ 261/1000 [26%] in 18s (14.6/s, eta: 51s) ▄▆█ 264/1000 [26%] in 18s (14.5/s, eta: 51s) ▁▃▅ 287/1000 [29%] in 19s (14.8/s, eta: 49s)  ▆█▆ 295/1000 [30%] in 20s (14.7/s, eta: 48s)  █▆▄ 298/1000 [30%] in 20s (14.7/s, eta: 48s)  ▃▁▃ 340/1000 [34%] in 23s (14.8/s, eta: 45s)  376/1000 [38%] in 25s (15.0/s, eta: 42s) 381/1000 [38%] in 25s (15.1/s, eta: 41s)  ▇▅▃ 392/1000 [39%] in 26s (15.1/s, eta: 40s) ▆▄▂ 395/1000 [40%] in 26s (15.1/s, eta: 40s)  ▅▃▁ 397/1000 [40%] in 26s (15.1/s, eta: 40s)  ▁▃▅ 404/1000 [40%] in 27s (15.1/s, eta: 40s)  ▇▇▅ 418/1000 [42%] in 28s (15.2/s, e

In [893]:
## Read transcripts in

#transcripts = []

#for index, trans in source.iterrows():
#    username = trans[USERNAME]
#    courseID = trans[COURSEID]
#    course_ver = trans[COURSEVERSION]
#    comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
#    status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
#    course_title = trans[COURSETITLE]
#    in_course, rep_ver = checkCourse(course_id, trans[COURSEID], trans[COURSEVERSION])
#    in_pers = checkPerson(people_un, trans[USERNAME])

    #Update Version
#    course_ver = rep_ver
    
#    trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
#    transcripts.append(trans_entry)
    #print(courseID, type(courseID), courseID in ids_only.values)

#print(len(transcripts))

### Print to Output

In [894]:
def write_row(df_output, trans, test=False, subset="ALL"):
    #df = df_output
    
    inCourse = trans.inCourse
    inPerson = trans.inPerson

    tmp_dict = {
        "LEARNER": trans.username,
        "COURSE": trans.courseID,
        "COURSE_VERSION": trans.courseVersion,
        "COMPLETION_DATE": trans.compDate,
        "COMPLETION_STATUS": trans.status,
        "CREATE_ADHOC_COURSE": "FALSE",
        "COURSE_TITLE": trans.courseTitle
    }
    if test == True:
        tmp_dict = {
            "LEARNER": trans.username,
            "COURSE": trans.courseID,
            "COURSE_VERSION": trans.courseVersion,
            "COMPLETION_DATE": trans.compDate,
            "COMPLETION_STATUS": trans.status,
            "CREATE_ADHOC_COURSE": "FALSE",
            "COURSE_TITLE": trans.courseTitle,
            "IN_COURSE": trans.inCourse,
            "IN_PERSON": trans.inPerson
        }
    if subset == "ALL":
        df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "CLEAN":
        if ( (inCourse == 1) & (inPerson == 1) ):
            df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "MISSING":
        if ( (inCourse == 0) | (inPerson == 0) ):
            tmp_dict["COURSE_GOOD"] = bool(inCourse)
            tmp_dict["PERSON_GOOD"] = bool(inPerson)
            df_output = df_output.append(tmp_dict, ignore_index=True)
    
    return df_output
    

In [895]:
# Output Values
out_header = [
    "LEARNER",
    "COURSE",
    "COURSE_VERSION",
    "COMPLETION_DATE",
    "COMPLETION_STATUS",
    "CREATE_ADHOC_COURSE",
    "COURSE_TITLE"
]

In [896]:
out = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out = write_row(out, ts, False, "CLEAN")
        bar()
out["ID"] = out.index
print(out.shape)
out.sample(10).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.2s (847.76/s)                                          
(379, 8)


,250,368,214,349,309,327,124,377,177,329
LEARNER,61195,92035,90242,60900,90435,53268,58328,57350,60317,57876
COURSE,IT-1019,OSH-1040,HR-1005,OSH-1007,MS-3001,EMP-1021-CRL,ERM-1002,IT-1028,OSH-9103,ENG-1072
COURSE_VERSION,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
COMPLETION_DATE,2021-10-26,2021-09-16,2021-11-08,2021-12-08,2021-02-03,2021-09-22,2021-06-09,2021-12-09,2022-03-15,2021-12-14
COMPLETION_STATUS,400,200,200,200,200,400,200,400,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,EDMS System Overview,Advanced Rigger,Mandatory Harassment and Violence Prevention T...,VIRTUAL Asbestos Module 6E,FD/ER - Integrated Work Control Practical Appl...,Officer in Charge and Emergency Steward - CRL,Land Use Program (LUP) Awareness,Documentation Lien Training,Mental Health First Aid (Virtual),Engineering Change Control Update –Ensure Impl...
ID,250,368,214,349,309,327,124,377,177,329


In [897]:
out.to_csv("out_transcript.csv", index=False)

### Write Failiures

In [898]:
out_err = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out_err = write_row(out_err, ts, False, "MISSING")
        bar()
out_err["ID"] = out_err.index
print(out_err.shape)
out_err.sample(10).T

|████████████████████████████████████████| 1000/1000 [100%] in 2.8s (353.12/s)                                          , eta: 1s) 
(621, 10)


,202,38,586,314,301,45,0,455,180,543
LEARNER,92627,91764,61865,61352,54238,90459,90525,54391,58228,56643
COURSE,OPEX-1005-Online (REV 1.4),LGL-1003-Online,FD-1004-Online,LGL-1003-Online (REV 3.3),LGL-1003-Online (Rev 2.5),OSH-3003-MULTI,OSH-9045-Online (Rev 1),RP-G3-Refresher-Online,CNL-1009-Online (REV 1. 3),S&T-3013
COURSE_VERSION,1.4,NaN,NaN,3.3,2.5,NaN,1.0,NaN,1.3,NaN
COMPLETION_DATE,2022-02-07,2022-01-25,2021-06-09,2022-03-01,2021-09-21,2021-07-22,2021-07-13,2021-09-23,2022-03-03,2021-11-16
COMPLETION_STATUS,200,200,200,200,200,200,200,400,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Introducing the ImpAct Process at CNL (1.4),Values and Ethics at CNL,Alpha Awareness,Values and Ethics at CNL (3.3),Values and Ethics at CNL (2.5),Aerial Platform - Practical (All Equipment Codes),Cost Course - Hand Safety (1.0),Group 3 Radiation Protection Refresher,Employee Formal Acknowledgement of CNL Executi...,Receiving Material into Building 405
COURSE_GOOD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PERSON_GOOD,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
ID,202,38,586,314,301,45,0,455,180,543


In [899]:
out_err.to_csv("out_errors.csv", index=False)

## Needed Courses and People

In [903]:
course_need = out_err[out_err["COURSE_GOOD"] == False]
person_need = out_err[out_err["PERSON_GOOD"] == False]

pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")

AttributeError: 'DataFrame' object has no attribute 'unique'